In [3]:
from pycaret.regression import *
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import *
from sklearn.kernel_ridge import KernelRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import *
from xgboost.sklearn import XGBRegressor
from lightgbm.sklearn import LGBMRegressor
from catboost.core import CatBoostRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, DotProduct

# MAE, MSE, RMSE
import math
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score, mean_absolute_percentage_error

In [4]:
def load_dataset(dataset):
    if dataset == 'ixi':
        ixi_train = pd.read_csv('./new_IXI/ixi_train.csv', index_col = 0)
        ixi_test = pd.read_csv('./new_IXI/ixi_test.csv', index_col = 0)

        # AGE에서 소수점 아래를 버림 -> for stratified k fold
        ixi_train['age'] = ixi_train['age'].astype('int64')
        ixi_test['age'] = ixi_test['age'].astype('int64')

        # 모든 feature의 dtype : float64
        ixi_train = ixi_train.astype('float64')
        ixi_test = ixi_test.astype('float64')
        return ixi_train, ixi_test
    
    elif dataset == 'hcp':
        hcp_train = pd.read_csv('./new_HCP/hcp_train.csv', index_col=0)
        hcp_test = pd.read_csv('./new_HCP/hcp_test.csv', index_col=0)

        hcp_train = hcp_train.astype('float64')
        hcp_test = hcp_test.astype('float64')
        return hcp_train, hcp_test
    
    elif dataset =='cc':
        cc_train = pd.read_csv('./new_CAMCAN/cc_train.csv', index_col = 0)
        cc_test = pd.read_csv('./new_CAMCAN/cc_test.csv', index_col = 0)

        cc_train = cc_train.astype('float64')
        cc_test = cc_test.astype('float64')
        return cc_train, cc_test

In [5]:
def load_pipelines(dataset):
    
    # Load Model
    load_ada = load_model(f'./models/{dataset}/best_model/AdaBoostRegressor')# 1
    load_ard = load_model(f'./models/{dataset}/best_model/ARDRegression')# 2
    load_br = load_model(f'./models/{dataset}/best_model/BayesianRidge')# 3
    load_dt = load_model(f'./models/{dataset}/best_model/DecisionTreeRegressor')# 4
    load_en = load_model(f'./models/{dataset}/best_model/ElasticNet')# 5
    load_et = load_model(f'./models/{dataset}/best_model/ExtraTreesRegressor')# 6
    load_gbr = load_model(f'./models/{dataset}/best_model/GradientBoostingRegressor')# 7
    load_hr = load_model(f'./models/{dataset}/best_model/HuberRegressor')# 8
    load_kr = load_model(f'./models/{dataset}/best_model/KernelRidge')# 9
    load_knn = load_model(f'./models/{dataset}/best_model/KNeighborsRegressor')# 10
    load_lars = load_model(f'./models/{dataset}/best_model/Lars')# 11
    load_lasso = load_model(f'./models/{dataset}/best_model/Lasso')# 12
    load_llar = load_model(f'./models/{dataset}/best_model/LassoLars')# 13
    load_lgbm = load_model(f'./models/{dataset}/best_model/LGBMRegressor')# 14
    load_lr = load_model(f'./models/{dataset}/best_model/LinearRegression')# 15
    load_mlp = load_model(f'./models/{dataset}/best_model/MLPRegressor')# 16
    load_omp = load_model(f'./models/{dataset}/best_model/OrthogonalMatchingPursuit')# 17
    load_par = load_model(f'./models/{dataset}/best_model/PassiveAggressiveRegressor')# 18
    load_rf = load_model(f'./models/{dataset}/best_model/RandomForestRegressor')# 19
    load_ransac = load_model(f'./models/{dataset}/best_model/RANSACRegressor')# 10
    load_ridge = load_model(f'./models/{dataset}/best_model/Ridge')# 21
    load_svr = load_model(f'./models/{dataset}/best_model/SVR')# 22
    load_xgbr = load_model(f'./models/{dataset}/best_model/XGBRegressor')# 23
    load_tr = load_model(f'./models/{dataset}/best_model/TheilSenRegressor')# 24
    load_catboost = load_model(f'./models/{dataset}/best_model/Catboost')# 25
    load_gp = load_model(f'./models/{dataset}/best_model/GaussianProcessRegressor') # 26




    model_list = [load_ada, load_ard, load_br, load_dt, load_en, load_et, load_gbr, load_hr, load_kr, load_knn, load_lars, load_lasso, load_llar, load_lgbm,
             load_lr, load_mlp, load_omp, load_par, load_rf, load_ransac, load_ridge, load_svr, load_xgbr, load_tr, load_catboost, load_gp]
    model_name_list = ['Adaboost', 'ARD','Bayesian Ridge', 'Decision Tree', 'ElasticNet', 'ExtraTreesRegressor','GradientBoostingRegressor','HuberRegressor','KernelRidge','KNeighborRegressor','Lars','Lasso',
                  'LassoLars','LGBMRegressor','LinearRegression','MLPRegressor','OrthogonalMatchingPursuit','PassiveAggressiveRegressor','RandomForestRegressor','RANSAC','Ridge','SVR','XGBRegressor', 
                  'TheilsenRegressor', 'Catboost','GaussianProcess']
    
    return model_list, model_name_list

In [6]:
# 기존에 ixi_test_score_df에서 test_df를 수정함으로써 전체 모델에 대해서 적용될 수 있도록 하였다. 
# 이후에, Test Score Grid에도 추가적인 Metric을 넣어주어야 할 때에는 같은 방식으로 코드를 수정하면 된다. 
def test_score_full_df(pipe_list, model_name_list, test_df):
    score_dict = {}
    mae_list= []
    mse_list = []
    rmse_list = []
    r2_list = []
    rmsle_list = []
    mape_list= []
    
    for pipeline in pipe_list:
        pred = pipeline.predict(test_df.drop('age', axis=1))
        mae_list.append(mean_absolute_error(test_df['age'], pred))
        mse_list.append(mean_squared_error(test_df['age'], pred))
        rmse_list.append(mean_squared_error(test_df['age'], pred, squared=False))
        r2_list.append(r2_score(test_df['age'], pred))
        rmsle_list.append(math.sqrt(mean_squared_log_error(test_df['age'], pred)))
        mape_list.append(mean_absolute_percentage_error(test_df['age'], pred))
        
    score_dict['Model'] = model_name_list
    score_dict['MAE'] = mae_list
    score_dict['MSE'] = mse_list
    score_dict['RMSE'] = rmse_list
    score_dict['R2'] = r2_list
    score_dict['RMSLE'] = rmsle_list
    score_dict['MAPE'] = mape_list
    
    score_df = pd.DataFrame(score_dict)
    score_df = score_df.sort_values('MAE').reset_index(drop=True)
    
    return score_df

In [9]:
ixi_train, ixi_test = load_dataset('ixi')
hcp_train, hcp_test = load_dataset('hcp')
cc_train, cc_test = load_dataset('cc')

In [10]:
ixi_pipe_list, ixi_pipe_name_list = load_pipelines('ixi')
hcp_pipe_list, hcp_pipe_name_list = load_pipelines('hcp')
cc_pipe_list, cc_pipe_name_list = load_pipelines('cc')

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and 

In [11]:
ixi_test_sg = test_score_full_df(ixi_pipe_list, ixi_pipe_name_list, ixi_test)
hcp_test_sg = test_score_full_df(hcp_pipe_list, hcp_pipe_name_list, hcp_test)
cc_test_sg = test_score_full_df(cc_pipe_list, cc_pipe_name_list, cc_test)

In [19]:
ixi_test_sg.to_csv('./new_IXI/ixi_best_model_test_score_full.csv')
hcp_test_sg.to_csv('./new_HCP/hcp_best_model_test_score_full.csv')
cc_test_sg.to_csv('./new_CAMCAN/cc_best_model_test_score_full.csv')

## Weighted MAE 값 구하기 

In [12]:
ixi_train_sg = pd.read_csv('./new_IXI/ixi_best_model_train_score.csv', index_col=0)
ixi_test_sg = pd.read_csv('./new_IXI/ixi_best_model_test_score_full.csv', index_col=0)

hcp_train_sg = pd.read_csv('./new_HCP/hcp_best_model_train_score.csv', index_col=0)
hcp_test_sg = pd.read_csv('./new_HCP/hcp_best_model_test_score_full.csv', index_col=0)

cc_train_sg = pd.read_csv('./new_CAMCAN/cc_best_model_train_score.csv', index_col=0)
cc_test_sg = pd.read_csv('./new_CAMCAN/cc_best_model_test_score_full.csv', index_col=0)

In [13]:
print(f'IXI Age Range : {ixi_train["age"].min()} ~ {ixi_train["age"].max()}')
print(f'HCP Age Range : {hcp_train["age"].min()} ~ {hcp_train["age"].max()}')
print(f'CAMCAN Age Range : {cc_train["age"].min()} ~ {cc_train["age"].max()}')

IXI Age Range : 20.0 ~ 86.0
HCP Age Range : 22.0 ~ 37.0
CAMCAN Age Range : 18.0 ~ 88.0


In [14]:
ixi_age_range = ixi_train['age'].max() - ixi_train['age'].min()

hcp_age_range = hcp_train['age'].max() - hcp_train['age'].min()

cc_age_range = cc_train['age'].max() - cc_train['age'].min()

In [15]:
ixi_age_range

66.0

In [16]:
hcp_age_range

15.0

In [17]:
cc_age_range

70.0

In [19]:
ixi_train_sg['wMAE'] = ixi_train_sg['MAE'] / ixi_age_range
ixi_test_sg['wMAE'] = ixi_test_sg['MAE'] / ixi_age_range

hcp_train_sg['wMAE'] = hcp_train_sg['MAE'] / hcp_age_range
hcp_test_sg['wMAE'] = hcp_test_sg['MAE'] / hcp_age_range

cc_train_sg['wMAE'] = cc_train_sg['MAE'] / cc_age_range
cc_test_sg['wMAE'] = cc_test_sg['MAE'] / cc_age_range


In [20]:
ixi_test_sg

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,R,wMAE
0,ARD,8.045330,95.229419,9.758556,0.639519,0.212433,0.183395,0.799787,0.121899
1,LassoLars,8.047342,96.166961,9.806475,0.635970,0.212457,0.183710,0.798141,0.121929
2,Lasso,8.047728,96.170512,9.806656,0.635957,0.212464,0.183719,0.798135,0.121935
3,MLPRegressor,8.067540,106.761510,10.332546,0.595866,0.228723,0.184012,0.777940,0.122235
4,PassiveAggressiveRegressor,8.236783,99.368746,9.968387,0.623850,0.217439,0.187883,0.790215,0.124800
5,Bayesian Ridge,8.278500,97.774439,9.888096,0.629885,0.215328,0.189066,0.794522,0.125432
6,GradientBoostingRegressor,8.315860,104.410728,10.218157,0.604764,0.220519,0.191696,0.781794,0.125998
7,ElasticNet,8.321723,98.405141,9.919937,0.627498,0.215616,0.190218,0.794730,0.126087
8,SVR,8.384531,99.945915,9.997295,0.621665,0.216789,0.191790,0.790417,0.127038
9,Ridge,8.391243,98.898633,9.944779,0.625630,0.216294,0.191801,0.793444,0.127140


In [21]:
hcp_test_sg

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,R,wMAE
0,Lasso,2.756452,11.172173,3.342480,0.177436,0.113543,0.097637,0.425805,0.183763
1,LassoLars,2.756465,11.172561,3.342538,0.177408,0.113545,0.097637,0.425783,0.183764
2,Lars,2.789603,11.535207,3.396352,0.150707,0.115174,0.098616,0.412385,0.185974
3,ElasticNet,2.791912,11.201596,3.346878,0.175270,0.113923,0.099229,0.419919,0.186127
4,Bayesian Ridge,2.792735,11.231557,3.351351,0.173064,0.114052,0.099235,0.418232,0.186182
5,Ridge,2.794133,11.124254,3.335304,0.180964,0.113620,0.099377,0.425521,0.186276
6,ARD,2.825067,11.937657,3.455091,0.121077,0.117114,0.099632,0.399124,0.188338
7,GradientBoostingRegressor,2.829546,11.063353,3.326162,0.185448,0.113080,0.100159,0.432620,0.188636
8,PassiveAggressiveRegressor,2.832213,11.868951,3.445134,0.126135,0.116886,0.100200,0.401049,0.188814
9,Catboost,2.832780,11.344769,3.368200,0.164729,0.114565,0.100440,0.406851,0.188852


In [22]:
cc_test_sg

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,R,wMAE
0,LassoLars,7.082988,80.247587,8.958102,0.735767,0.179715,0.145547,0.858858,0.101186
1,ARD,7.103989,80.084797,8.949011,0.736303,0.180544,0.147479,0.858484,0.101486
2,Lasso,7.175729,81.166221,9.009230,0.732742,0.180200,0.146888,0.856730,0.102510
3,ElasticNet,7.181621,82.276601,9.070645,0.729086,0.181173,0.147660,0.854815,0.102595
4,HuberRegressor,7.466341,86.798269,9.316559,0.714197,0.185665,0.153975,0.845451,0.106662
5,Bayesian Ridge,7.469777,87.346248,9.345921,0.712393,0.186065,0.154612,0.844482,0.106711
6,PassiveAggressiveRegressor,7.576236,90.018374,9.487801,0.703594,0.187609,0.155020,0.839472,0.108232
7,Ridge,7.586470,87.918141,9.376467,0.710510,0.188287,0.156574,0.843236,0.108378
8,OrthogonalMatchingPursuit,7.617903,88.123884,9.387432,0.709832,0.186118,0.155676,0.843671,0.108827
9,GaussianProcess,7.719002,91.252557,9.552620,0.699531,0.193282,0.159210,0.837652,0.110271


In [24]:
ixi_test_sg.to_csv('./new_IXI/ixi_best_model_test_score_full.csv')
hcp_test_sg.to_csv('./new_HCP/hcp_best_model_test_score_full.csv')
cc_test_sg.to_csv('./new_CAMCAN/cc_best_model_test_score_full.csv')

In [54]:
ixi_train_sg = ixi_train_sg[['Model', 'MAE', 'MSE', 'RMSE', 'R2','RMSLE','MAPE', 'wMAE', 'TT (Sec)']]
hcp_train_sg = hcp_train_sg[['Model', 'MAE', 'MSE', 'RMSE', 'R2','RMSLE','MAPE', 'wMAE', 'TT (Sec)']]
cc_train_sg = cc_train_sg[['Model', 'MAE', 'MSE', 'RMSE', 'R2','RMSLE','MAPE', 'wMAE', 'TT (Sec)']]

In [56]:
ixi_train_sg.to_csv('./new_IXI/ixi_best_model_train_score_full.csv')
hcp_train_sg.to_csv('./new_HCP/hcp_best_model_train_score_full.csv')
cc_train_sg.to_csv('./new_CAMCAN/cc_best_model_train_score_full.csv')

In [25]:
ixi_test_sg_model_sort = ixi_test_sg.sort_values('Model')
hcp_test_sg_model_sort = hcp_test_sg.sort_values('Model')
cc_test_sg_model_sort = cc_test_sg.sort_values('Model')

In [26]:
ixi_test_sg_model_sort.to_csv('./ixi_test_sg_model.csv')
hcp_test_sg_model_sort.to_csv('./hcp_test_sg_model.csv')
cc_test_sg_model_sort.to_csv('./cc_test_sg_model.csv')

## Person Correlation 구하기 

In [60]:
ixi_train_sg = pd.read_csv('./new_IXI/ixi_best_model_train_score_full.csv', index_col=0)
ixi_test_sg = pd.read_csv('./new_IXI/ixi_best_model_test_score_full.csv', index_col=0)

hcp_train_sg = pd.read_csv('./new_HCP/hcp_best_model_train_score_full.csv', index_col=0)
hcp_test_sg = pd.read_csv('./new_HCP/hcp_best_model_test_score_full.csv', index_col=0)

cc_train_sg = pd.read_csv('./new_CAMCAN/cc_best_model_train_score_full.csv', index_col=0)
cc_test_sg = pd.read_csv('./new_CAMCAN/cc_best_model_test_score_full.csv', index_col=0)

In [62]:
# 기존에 ixi_test_score_df에서 test_df를 수정함으로써 전체 모델에 대해서 적용될 수 있도록 하였다. 
# 이후에, Test Score Grid에도 추가적인 Metric을 넣어주어야 할 때에는 같은 방식으로 코드를 수정하면 된다. 
def test_score_include_corr(pipe_list, model_name_list, test_df):
    score_dict = {}
    mae_list= []
    mse_list = []
    rmse_list = []
    r2_list = []
    rmsle_list = []
    mape_list= []
    corr_list = []
    
    for pipeline in pipe_list:
        pred = pipeline.predict(test_df.drop('age', axis=1))
        mae_list.append(mean_absolute_error(test_df['age'], pred))
        mse_list.append(mean_squared_error(test_df['age'], pred))
        rmse_list.append(mean_squared_error(test_df['age'], pred, squared=False))
        r2_list.append(r2_score(test_df['age'], pred))
        rmsle_list.append(math.sqrt(mean_squared_log_error(test_df['age'], pred)))
        mape_list.append(mean_absolute_percentage_error(test_df['age'], pred))
        corr_list.append(np.corrcoef(test_df['age'], pred)[0,1])
        
    score_dict['Model'] = model_name_list
    score_dict['MAE'] = mae_list
    score_dict['MSE'] = mse_list
    score_dict['RMSE'] = rmse_list
    score_dict['R2'] = r2_list
    score_dict['RMSLE'] = rmsle_list
    score_dict['MAPE'] = mape_list
    score_dict['R'] = corr_list
    
    score_df = pd.DataFrame(score_dict)
    score_df = score_df.sort_values('MAE').reset_index(drop=True)
    
    return score_df

In [65]:
ixi_test_corr = test_score_include_corr(ixi_pipe_list, ixi_pipe_name_list, ixi_test)
ixi_train_corr = test_score_include_corr(ixi_pipe_list, ixi_pipe_name_list, ixi_train)

hcp_test_corr = test_score_include_corr(hcp_pipe_list, hcp_pipe_name_list, hcp_test)
hcp_train_corr = test_score_include_corr(hcp_pipe_list, hcp_pipe_name_list, hcp_train)

cc_test_corr = test_score_include_corr(cc_pipe_list, cc_pipe_name_list, cc_test)
cc_train_corr = test_score_include_corr(cc_pipe_list, cc_pipe_name_list, cc_train)

In [71]:
ixi_test_corr.to_csv('./new_IXI/ixi_best_model_test_score_full.csv')
hcp_test_corr.to_csv('./new_HCP/hcp_best_model_test_score_full.csv')
cc_test_corr.to_csv('./new_CAMCAN/cc_best_model_test_score_full.csv')

In [76]:
ixi_train_sg['R'] = ixi_train_sg['R2'].apply(lambda x : math.sqrt(x))
hcp_train_sg['R'] = hcp_train_sg['R2'].apply(lambda x : math.sqrt(x))
cc_train_sg['R'] = cc_train_sg['R2'].apply(lambda x : math.sqrt(x))

In [78]:
ixi_train_sg = ixi_train_sg[['Model', 'MAE','MSE','RMSE','R2','RMSLE','MAPE','wMAE','R','TT (Sec)']]
hcp_train_sg = hcp_train_sg[['Model', 'MAE','MSE','RMSE','R2','RMSLE','MAPE','wMAE','R','TT (Sec)']]
cc_train_sg = cc_train_sg[['Model', 'MAE','MSE','RMSE','R2','RMSLE','MAPE','wMAE','R','TT (Sec)']]

In [80]:
ixi_train_sg.to_csv('./new_IXI/ixi_best_model_train_score_full.csv')
hcp_train_sg.to_csv('./new_HCP/hcp_best_model_train_score_full.csv')
cc_train_sg.to_csv('./new_CAMCAN/cc_best_model_train_score_full.csv')

In [89]:
ixi_train_sg = pd.read_csv('./new_IXI/ixi_best_model_train_score_full.csv', index_col=0)
ixi_test_sg = pd.read_csv('./new_IXI/ixi_best_model_test_score_full.csv', index_col=0)

hcp_train_sg = pd.read_csv('./new_HCP/hcp_best_model_train_score_full.csv', index_col=0)
hcp_test_sg = pd.read_csv('./new_HCP/hcp_best_model_test_score_full.csv', index_col=0)

cc_train_sg = pd.read_csv('./new_CAMCAN/cc_best_model_train_score_full.csv', index_col=0)
cc_test_sg = pd.read_csv('./new_CAMCAN/cc_best_model_test_score_full.csv', index_col=0)

In [86]:
ixi_train_sg = ixi_train_sg.sort_values('Model').reset_index(drop=True)
ixi_test_sg = ixi_test_sg.sort_values('Model').reset_index(drop=True)

hcp_train_sg = hcp_train_sg.sort_values('Model').reset_index(drop=True)
hcp_test_sg = hcp_test_sg.sort_values('Model').reset_index(drop=True)

cc_train_sg = cc_train_sg.sort_values('Model').reset_index(drop=True)
cc_test_sg = cc_test_sg.sort_values('Model').reset_index(drop=True)

In [88]:
ixi_train_sg

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,wMAE,R,TT (Sec)
0,AdaBoost Regressor,7.7603,90.8845,9.4691,0.6584,0.2213,0.1922,0.117580,0.811419,1.468
1,Automatic Relevance Determination,7.4790,87.2582,9.3023,0.6735,0.2207,0.1817,0.113318,0.820670,0.222
2,Bayesian Ridge,7.4376,85.4214,9.2152,0.6813,0.2155,0.1808,0.112691,0.825409,0.070
3,CatBoost Regressor,7.6574,86.5556,9.2785,0.6763,0.2148,0.1879,0.116021,0.822375,5.534
4,Decision Tree Regressor,9.3118,151.6218,12.2431,0.4353,0.2767,0.2257,0.141088,0.659773,0.135
5,Elastic Net,7.3865,83.6484,9.1223,0.6866,0.2140,0.1809,0.111917,0.828613,0.016
6,Extra Trees Regressor,7.7683,91.3625,9.5285,0.6584,0.2234,0.1949,0.117702,0.811419,0.883
7,Extreme Gradient Boosting,7.4845,87.4288,9.3209,0.6720,0.2130,0.1816,0.113402,0.819756,0.273
8,GaussianProcessRegressor,8.4452,111.4857,10.5033,0.5813,0.2560,0.2031,0.127958,0.762430,0.031
9,Gradient Boosting Regressor,7.5873,91.8074,9.5333,0.6590,0.2181,0.1854,0.114959,0.811788,0.876


In [87]:
ixi_test_sg

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,R
0,ARD,8.045330,95.229419,9.758556,0.639519,0.212433,0.183395,0.799787
1,Adaboost,9.236637,121.728733,11.033075,0.539209,0.242225,0.213759,0.740247
2,Bayesian Ridge,8.278500,97.774439,9.888096,0.629885,0.215328,0.189066,0.794522
3,Catboost,8.608453,104.117758,10.203811,0.605873,0.225563,0.200345,0.791841
4,Decision Tree,9.750000,167.572368,12.944975,0.365673,0.285315,0.231252,0.645655
5,ElasticNet,8.321723,98.405141,9.919937,0.627498,0.215616,0.190218,0.794730
6,ExtraTreesRegressor,8.857753,111.236561,10.546874,0.578926,0.232574,0.207167,0.782321
7,GaussianProcess,8.392527,109.551794,10.466699,0.585303,0.231070,0.191935,0.771927
8,GradientBoostingRegressor,8.315860,104.410728,10.218157,0.604764,0.220519,0.191696,0.781794
9,HuberRegressor,8.515712,110.722463,10.522474,0.580872,0.234266,0.196565,0.770429
